In [1]:
# import neccessary dependencies
from citipy import citipy
import gmaps
from config import g_key,weather_api_key
import numpy as np
import pandas as pd
import requests

In [2]:
#generate 2000 random lats and longs
lats = np.random.uniform(-90,90,size=2000)
long = np.random.uniform(-180,180,size=2000)
coords = list(zip(lats,long))

In [3]:
# get the nearest cities for each coordinate pair 
cities = []
for coordinate in coords:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
print(len(cities))
len(cities)

759


759

In [4]:
# create a DataFrame with the city names
cities_df = pd.DataFrame(cities)
cities_df.columns = ["City"]

# define a function that will be applied to the df which calls the api and fills out the fields
def get_weather(row):
    city = row["City"]
    base_url = "https://api.openweathermap.org/data/2.5/weather?"
    params = {
        'units' : "imperial",
        'q' : city,
        'appid' : weather_api_key
    }
    try :
        print(f"Processing | {city}")
        r = requests.get(base_url,params).json()
        row["Lat"] = r["coord"]["lat"]
        row["Lng"] = r["coord"]["lon"]
        row["Country"] = r["sys"]["country"]
        row["Max Temp"] = r["main"]["temp_max"]
        row["% Humidity"] = r["main"]["humidity"]
        row["% Cloudiness"] = r["clouds"]["all"]
        row["Wind Speed"] = r["wind"]["speed"]
        row["Weather Description"] = r["weather"][0]["description"]
    except KeyError as keyEx:
        print("City not found... skipping")
    return row
# apply the function to the dataFrame
cities_df = cities_df.apply(get_weather, axis=1)
#drop any rows that were not found
cities_df = cities_df.dropna()
cities_df.head()

Processing | nishihara
Processing | nishihara
Processing | vaini
Processing | faya
Processing | barentsburg
City not found... skipping
Processing | ushuaia
Processing | berlevag
Processing | cape town
Processing | busselton
Processing | westport
Processing | carnarvon
Processing | coquimbo
Processing | norman wells
Processing | rikitea
Processing | harper
Processing | ponta do sol
Processing | punta arenas
Processing | cherskiy
Processing | butaritari
Processing | lodingen
Processing | namatanai
Processing | tamiahua
Processing | uvalde
Processing | half moon bay
Processing | cicero dantas
Processing | yatou
Processing | cernay
Processing | berovo
Processing | albany
Processing | port alfred
Processing | hermanus
Processing | mwene-ditu
Processing | aasiaat
Processing | gat
Processing | port elizabeth
Processing | port augusta
Processing | qaanaaq
Processing | devils lake
Processing | foligno
Processing | yellowknife
Processing | tilichiki
Processing | igarka
Processing | dikson
Proces

Processing | braco do norte
Processing | korla
Processing | khilok
Processing | mount isa
Processing | lingyuan
Processing | shingu
Processing | lodi
Processing | boden
Processing | teya
Processing | dolbeau
City not found... skipping
Processing | tahoua
Processing | arlit
Processing | malaga
Processing | minas novas
Processing | abaza
Processing | kampong thum
Processing | kambove
Processing | karratha
Processing | nantucket
Processing | karaul
City not found... skipping
Processing | iracoubo
Processing | omboue
Processing | kamenka
Processing | auch
Processing | omsukchan
Processing | meulaboh
Processing | baracoa
Processing | warmbad
Processing | gawler
Processing | gashua
Processing | tigil
Processing | iguape
Processing | samusu
City not found... skipping
Processing | lazaro cardenas
Processing | baykit
Processing | prince rupert
Processing | cairns
Processing | uroteppa
City not found... skipping
Processing | cabedelo
Processing | mayo
Processing | sechura
Processing | scottsbluf

Processing | bongor
Processing | sestri levante
Processing | pringsewu
Processing | ozatlan
Processing | marfino
Processing | camacha
Processing | watsa
Processing | galle
Processing | general roca
Processing | sibolga
Processing | broome
Processing | pimenta bueno
Processing | petatlan
Processing | changji
Processing | luanda
Processing | handan
Processing | moerai
Processing | bassano
Processing | gallikos
Processing | tokur
Processing | vallenar
Processing | hami
Processing | boundiali
Processing | aba
Processing | pousat
City not found... skipping
Processing | bermeo
Processing | dunda
Processing | balezino
Processing | uyuni
Processing | vilhena
Processing | pingxiang
Processing | agadez
Processing | bairiki
Processing | klaksvik
Processing | fare
Processing | biscarrosse
Processing | marawi
Processing | tsabong
Processing | khandyga
Processing | paamiut
Processing | suoyarvi
Processing | sisimiut
Processing | nefteyugansk
Processing | kondinskoye
Processing | carberry
Processing 

,% Cloudiness,% Humidity,City,Country,Lat,Lng,Max Temp,Weather Description,Wind Speed
0,40.0,77.0,nishihara,JP,26.1842,127.7558,68.0,scattered clouds,12.66
1,75.0,88.0,vaini,TO,-21.2000,-175.2000,73.4,broken clouds,4.61
2,20.0,43.0,faya,SA,18.3851,42.4509,75.2,few clouds,12.66
4,75.0,58.0,ushuaia,AR,-54.8000,-68.3000,51.8,broken clouds,6.91
5,40.0,69.0,berlevag,NO,70.8578,29.0864,33.8,scattered clouds,13.80


In [9]:
cities_df = cities_df[["City","Country","Lat","Lng","Max Temp","% Humidity","% Cloudiness","Wind Speed","Weather Description"]]
cities_df.to_csv("WeatherPy_Database.csv")